<a href="https://colab.research.google.com/github/alimomennasab/ChestXRay-Classification/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pandas as pd
import os
import numpy as np
from PIL import Image
from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##2. Loading Data

In [ ]:
data_dir = '/content/drive/My Drive/chest_xray/'

In [ ]:
# Split dataset into training, validation, and test sets
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [ ]:
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'chest_xray', 'val', '__MACOSX', 'test']
['PNEUMONIA', 'NORMAL']


In [ ]:
# Pneumonia images
pneumonia_files = os.listdir(data_dir + "/train/PNEUMONIA")
print('No. of training examples for Pneumonia:', len(pneumonia_files))
print(pneumonia_files[:5])

No. of training examples for Pneumonia: 3875
['person556_virus_1096.jpeg', 'person536_bacteria_2257.jpeg', 'person581_bacteria_2390.jpeg', 'person592_bacteria_2434.jpeg', 'person581_virus_1125.jpeg']


In [ ]:
# Normal (healthy) images
normal_files = os.listdir(data_dir + "/train/NORMAL")
print('No. of training examples for Normal:', len(normal_files))
print(normal_files[:5])

No. of training examples for Normal: 1341
['IM-0526-0001.jpeg', 'IM-0524-0001.jpeg', 'IM-0507-0001.jpeg', 'IM-0508-0001.jpeg', 'IM-0520-0001.jpeg']


In [ ]:
# There are almost three times more pneumonia images than normal images, so we will use class weighing

# Define classes
classes = ['NORMAL', 'PNEUMONIA']

# Define class weights
num_pneumonia_train = len(os.listdir(os.path.join(train_dir, classes[1])))
num_normal_train = len(os.listdir(os.path.join(train_dir, classes[0])))
total_train = num_pneumonia_train + num_normal_train
class_weights = torch.tensor([total_train/num_normal_train, total_train/num_pneumonia_train]).to(device)

##3. Preparing Dataset and DataLoader

In [ ]:
# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

val_and_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [ ]:
# Define data directories. We won't use a custom class because the dataset is already well-formatted.

train_dataset = ImageFolder('/content/drive/My Drive/chest_xray/train', transform = train_transform)
val_dataset = ImageFolder('/content/drive/My Drive/chest_xray/val', transform = val_and_test_transform)
test_dataset = ImageFolder('/content/drive/My Drive/chest_xray/test', transform = val_and_test_transform)

train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = False)

##4. Defining and Choosing Model

In [ ]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride = 1):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

class ResNet(nn.Module): # [3, 4, 6, 3]
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #ResNet layers
        self.layer1 = self._make_layer(block, layers[0], out_channels=64, stride=1)
        self.layer2 = self._make_layer(block, layers[1], out_channels=128, stride=2)
        self.layer3 = self._make_layer(block, layers[2], out_channels=256, stride=2)
        self.layer4 = self._make_layer(block, layers[3], out_channels=512, stride=2) #2048 channels at end

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def _make_layer(self, block, num_residual_blocks, out_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != out_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels*4, kernel_size=1,
                                                          stride=stride),
                                                nn.BatchNorm2d(out_channels*4))

        layers.append(block(self.in_channels, out_channels, identity_downsample, stride))
        self.in_channels = out_channels*4 #256 in channels

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, out_channels)) # 256 in channels, 64 out channels. 256 -> 64, 64*4 (256) again

        return nn.Sequential(*layers)


def ResNet50(img_channels=3, num_classes=2):
    return ResNet(block, [3, 4, 6, 3], img_channels, num_classes)

def ResNet101(img_channels=3, num_classes=2):
    return ResNet(block, [3, 4, 23, 3], img_channels, num_classes)

def ResNet152(img_channels=3, num_classes=2):
    return ResNet(block, [3, 8, 36, 3], img_channels, num_classes)


In [ ]:
#Choosing ResNet type
model = ResNet50()
#model = ResNet101()
#model = ResNet152()

In [ ]:
if torch.cuda.is_available():
    model.cuda()

##5. Testing Model

In [ ]:
def test():
    net = ResNet152()
    x = torch.randn(2, 3, 224, 224)
    y = net(x).to('cpu') #change to cuda
    print(y.shape) 

In [ ]:
test()

torch.Size([2, 2])


##6. Defining Main Training

In [ ]:
# Hyper-parameters
num_epochs = 30
learning_rate = 0.001
patience = 10

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

curr_lr = learning_rate
total_step = len(train_loader)

# Early stopping parameters
early_stopping_counter = 0
best_loss = float('inf')

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

    # Calculate validation loss
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels)
        val_loss /= len(val_loader)

        # Save the model if the validation loss is the best observed yet
        if val_loss < best_loss:
            print(f'Saving model with validation loss of {val_loss:.4f}...')
            torch.save(model.state_dict(), 'best_model.pth')
            best_loss = val_loss

    # Early stopping if overfitting
    if early_stopping_counter >= patience:
        print(f'Validation loss has not improved for {patience} epochs. Early stopping...')
        break
    elif val_loss < best_loss:
        best_loss = val_loss
        early_stopping_counter = 0


Epoch [1/30], Step [100/652] Loss: 0.9314
Epoch [1/30], Step [200/652] Loss: 0.5341
Epoch [1/30], Step [300/652] Loss: 0.3641
Epoch [1/30], Step [400/652] Loss: 0.7118
Epoch [1/30], Step [500/652] Loss: 0.6268
Epoch [1/30], Step [600/652] Loss: 0.6328
Saving model with validation loss of 0.7317...
Epoch [2/30], Step [100/652] Loss: 0.8648
Epoch [2/30], Step [200/652] Loss: 1.0601
Epoch [2/30], Step [300/652] Loss: 0.7649
Epoch [2/30], Step [400/652] Loss: 0.7003
Epoch [2/30], Step [500/652] Loss: 0.7160
Epoch [2/30], Step [600/652] Loss: 0.5637
Saving model with validation loss of 0.7097...
Epoch [3/30], Step [100/652] Loss: 0.6370
Epoch [3/30], Step [200/652] Loss: 0.6532
Epoch [3/30], Step [300/652] Loss: 0.7382
Epoch [3/30], Step [400/652] Loss: 0.4637
Epoch [3/30], Step [500/652] Loss: 0.7120
Epoch [3/30], Step [600/652] Loss: 0.6293
Epoch [4/30], Step [100/652] Loss: 0.5432
Epoch [4/30], Step [200/652] Loss: 0.6032
Epoch [4/30], Step [300/652] Loss: 0.4208
Epoch [4/30], Step [400/

##7. Testing

In [ ]:
# Load the saved model checkpoint
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Model accuracy on test images: {} %'.format(100 * correct / total))

Model accuracy on test images: 72.27564102564102 %
